In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/Reject/2020-02-04 05_24_35.900678.jpg
/kaggle/input/Reject/2020-02-04 05_20_59.011264.jpg
/kaggle/input/Reject/2020-02-04 05_20_22.514603.jpg
/kaggle/input/Reject/2020-02-04 05_18_16.123743.jpg
/kaggle/input/Bottle_200cl/image2020-02-18 00_26_34.098978.jpg
/kaggle/input/Bottle_200cl/image2020-02-18 00_15_18.780327.jpg
/kaggle/input/Bottle_200cl/image2020-02-18 00_21_36.270727.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_23_49.727559.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_22_39.251346.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_24_49.949408.jpg
/kaggle/input/Bottle_200cl/image2020-02-18 00_32_00.557065.jpg
/kaggle/input/Bottle_200cl/2020-02-04 04_46_11.380607.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_22_52.331052.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_20_00.599622.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_19_14.290297.jpg
/kaggle/input/Bottle_200cl/2020-02-04 05_21_37.704887.jpg
/kaggle/input/Bottle_200cl/image2020-02-18 00_21_19.754833.jpg
/kaggle/input

The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [3]:
# Distribution graphs (histogram/bar graph) of column data
def plot_per_column_dist(df, n_graph_shown, n_graph_per_row):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    n_rows, n_cols = df.shape
    column_names = list(df)
    n_graph_row = (n_cols + n_graph_per_row - 1) / n_graph_per_row
    plt.figure(num = None, figsize = (6 * n_graph_per_row, 8 * n_graph_row), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(n_cols, n_graph_shown)):
        plt.subplot(n_graph_row, n_graph_per_row, i + 1)
        columns_df = df.iloc[:, i]
        if (not np.issubdtype(type(columns_df.iloc[0]), np.number)):
            valueCounts = columns_df.value_counts()
            valueCounts.plot.bar()
        else:
            columns_df.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{column_names[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [4]:
# Correlation matrix
def plot_corr_mat(df, graph_width):
    file_name = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graph_width, graph_width), dpi=80, facecolor='w', edgecolor='k')
    corr_matrix = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corr_matrix)
    plt.title(f'Correlation Matrix for {file_name}', fontsize=15)
    plt.show()

In [5]:
# Scatter and density plots
def plot_scatter_mat(df, plot_size, text_size):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    ## dropping null values
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    column_names = list(df)
    if len(column_names) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        column_names = column_names[:10]
    df = df[column_names]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plot_size, plot_size], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=text_size)
    plt.suptitle('Scatter and Density Plot')
    plt.show()